In [22]:
import pandas as pd
import numpy as np
import yfinance as yf
import FinanceDataReader as fdr
import json

In [12]:
yf.Ticker('META').history()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2023-07-17 00:00:00-04:00,307.540009,311.709991,304.709991,310.619995,25323100,0.0,0.0
2023-07-18 00:00:00-04:00,310.880005,314.200012,307.619995,312.049988,20764600,0.0,0.0
2023-07-19 00:00:00-04:00,313.029999,318.679993,310.519989,316.010010,21763700,0.0,0.0
2023-07-20 00:00:00-04:00,313.500000,315.540009,302.220001,302.519989,23836900,0.0,0.0
2023-07-21 00:00:00-04:00,304.570007,305.459991,291.200012,294.260010,42089700,0.0,0.0
2023-07-24 00:00:00-04:00,295.779999,297.519989,288.299988,291.609985,24915700,0.0,0.0
2023-07-25 00:00:00-04:00,295.190002,298.299988,291.859985,294.470001,19585600,0.0,0.0
2023-07-26 00:00:00-04:00,301.190002,301.769989,291.899994,298.570007,47256900,0.0,0.0
2023-07-27 00:00:00-04:00,325.119995,325.350006,309.839996,311.709991,64229200,0.0,0.0


In [46]:
ticker = ['^IXIC', 'AAPL', 'MSFT', 'META']
data = []
for t in ticker:
    df = yf.Ticker(t).history()
    df.index = df.index.map(lambda x: str(x)[:10])
    data.append(json.loads(df.to_json(orient='index')))
data = json.dumps(data, indent=2)
data = json.loads(data)
with open('./data.json', 'w') as file:
    json.dump(data, file, indent=2)

In [28]:
output

'["{\\"2023-07-17\\":{\\"Open\\":14149.919921875,\\"High\\":14274.41015625,\\"Low\\":14138.009765625,\\"Close\\":14244.9501953125,\\"Volume\\":4593160000,\\"Dividends\\":0.0,\\"Stock Splits\\":0.0},\\"2023-07-18\\":{\\"Open\\":14212.25,\\"High\\":14396.6904296875,\\"Low\\":14176.1103515625,\\"Close\\":14353.6396484375,\\"Volume\\":4824070000,\\"Dividends\\":0.0,\\"Stock Splits\\":0.0},\\"2023-07-19\\":{\\"Open\\":14398.5302734375,\\"High\\":14446.5498046875,\\"Low\\":14317.08984375,\\"Close\\":14358.01953125,\\"Volume\\":5112420000,\\"Dividends\\":0.0,\\"Stock Splits\\":0.0},\\"2023-07-20\\":{\\"Open\\":14273.2900390625,\\"High\\":14309.919921875,\\"Low\\":14030.6201171875,\\"Close\\":14063.3095703125,\\"Volume\\":5128020000,\\"Dividends\\":0.0,\\"Stock Splits\\":0.0},\\"2023-07-21\\":{\\"Open\\":14148.1796875,\\"High\\":14179.009765625,\\"Low\\":14020.4501953125,\\"Close\\":14032.8095703125,\\"Volume\\":5254180000,\\"Dividends\\":0.0,\\"Stock Splits\\":0.0},\\"2023-07-24\\":{\\"Open\\